In [2]:
import rdflib
from rdflib import Graph, Literal, RDF, RDFS, URIRef, Namespace
from rdflib import Graph
from rdflib.namespace import FOAF, XSD
import pandas as pd
import numpy as np
import re 
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [3]:
t_list = []

#Top 250 of all films
#Top 50 films of each IMDb category

addresses = [
    "https://www.imdb.com/chart/top/",
    "https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action&sort=user_rating,desc&start=1&ref_=adv_nxt",
    "https://www.imdb.com/search/title/?genres=animation&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_3",
    "https://www.imdb.com/search/title/?genres=biography&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_4",
    "https://www.imdb.com/search/title/?genres=comedy&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_5",
    "https://www.imdb.com/search/title/?genres=crime&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_6",
    "https://www.imdb.com/search/title/?genres=drama&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_7",
    "https://www.imdb.com/search/title/?genres=family&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_8",
    "https://www.imdb.com/search/title/?genres=fantasy&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_9",
    "https://www.imdb.com/search/title/?genres=film_noir&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_10",
    "https://www.imdb.com/search/title/?genres=history&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_11",
    "https://www.imdb.com/search/title/?genres=horror&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_12",
    "https://www.imdb.com/search/title/?genres=music&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_13",
    "https://www.imdb.com/search/title/?genres=musical&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_14",
    "https://www.imdb.com/search/title/?genres=mystery&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_15",
    "https://www.imdb.com/search/title/?genres=romance&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_16",
    "https://www.imdb.com/search/title/?genres=sci_fi&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_17",
    "https://www.imdb.com/search/title/?genres=sport&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_18",
    "https://www.imdb.com/search/title/?genres=thriller&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_19",
    "https://www.imdb.com/search/title/?genres=western&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_21",
    "https://www.imdb.com/search/title/?genres=western&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=V5K8QY2XKQYH6NA8WJ5V&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_21",
]

for address in addresses:

    source = requests.get(address)
    source.raise_for_status()
    soup = BeautifulSoup(source.text, "html.parser")
    movies = soup.find_all('a')

    for m in tqdm(movies):
        href = m.get('href')
        if href != None:
            s = href.split("/")
            try:
                if s[1] == "title":
                    t_list += [s[2]]
            except Exception as e:
                pass

t_list

100%|████████████████████████████████████████████████████████████████████████████████████████| 983/983 [00:00<?, ?it/s]


['tt0111161',
 'tt0111161',
 'tt0068646',
 'tt0068646',
 'tt0468569',
 'tt0468569',
 'tt0071562',
 'tt0071562',
 'tt0050083',
 'tt0050083',
 'tt0108052',
 'tt0108052',
 'tt0167260',
 'tt0167260',
 'tt0110912',
 'tt0110912',
 'tt0120737',
 'tt0120737',
 'tt0060196',
 'tt0060196',
 'tt0109830',
 'tt0109830',
 'tt0137523',
 'tt0137523',
 'tt0167261',
 'tt0167261',
 'tt1375666',
 'tt1375666',
 'tt0080684',
 'tt0080684',
 'tt0133093',
 'tt0133093',
 'tt0099685',
 'tt0099685',
 'tt0073486',
 'tt0073486',
 'tt0114369',
 'tt0114369',
 'tt0047478',
 'tt0047478',
 'tt0038650',
 'tt0038650',
 'tt0102926',
 'tt0102926',
 'tt0120815',
 'tt0120815',
 'tt0317248',
 'tt0317248',
 'tt0816692',
 'tt0816692',
 'tt0118799',
 'tt0118799',
 'tt0120689',
 'tt0120689',
 'tt0076759',
 'tt0076759',
 'tt0103064',
 'tt0103064',
 'tt0088763',
 'tt0088763',
 'tt0245429',
 'tt0245429',
 'tt0253474',
 'tt0253474',
 'tt0054215',
 'tt0054215',
 'tt6751668',
 'tt6751668',
 'tt0110413',
 'tt0110413',
 'tt0110357',
 'tt01

In [4]:
t_list

['tt0111161',
 'tt0111161',
 'tt0068646',
 'tt0068646',
 'tt0468569',
 'tt0468569',
 'tt0071562',
 'tt0071562',
 'tt0050083',
 'tt0050083',
 'tt0108052',
 'tt0108052',
 'tt0167260',
 'tt0167260',
 'tt0110912',
 'tt0110912',
 'tt0120737',
 'tt0120737',
 'tt0060196',
 'tt0060196',
 'tt0109830',
 'tt0109830',
 'tt0137523',
 'tt0137523',
 'tt0167261',
 'tt0167261',
 'tt1375666',
 'tt1375666',
 'tt0080684',
 'tt0080684',
 'tt0133093',
 'tt0133093',
 'tt0099685',
 'tt0099685',
 'tt0073486',
 'tt0073486',
 'tt0114369',
 'tt0114369',
 'tt0047478',
 'tt0047478',
 'tt0038650',
 'tt0038650',
 'tt0102926',
 'tt0102926',
 'tt0120815',
 'tt0120815',
 'tt0317248',
 'tt0317248',
 'tt0816692',
 'tt0816692',
 'tt0118799',
 'tt0118799',
 'tt0120689',
 'tt0120689',
 'tt0076759',
 'tt0076759',
 'tt0103064',
 'tt0103064',
 'tt0088763',
 'tt0088763',
 'tt0245429',
 'tt0245429',
 'tt0253474',
 'tt0253474',
 'tt0054215',
 'tt0054215',
 'tt6751668',
 'tt6751668',
 'tt0110413',
 'tt0110413',
 'tt0110357',
 'tt01

In [5]:
len(t_list)

3455

In [6]:
unique_key = list(dict.fromkeys(t_list))
unique_key

['tt0111161',
 'tt0068646',
 'tt0468569',
 'tt0071562',
 'tt0050083',
 'tt0108052',
 'tt0167260',
 'tt0110912',
 'tt0120737',
 'tt0060196',
 'tt0109830',
 'tt0137523',
 'tt0167261',
 'tt1375666',
 'tt0080684',
 'tt0133093',
 'tt0099685',
 'tt0073486',
 'tt0114369',
 'tt0047478',
 'tt0038650',
 'tt0102926',
 'tt0120815',
 'tt0317248',
 'tt0816692',
 'tt0118799',
 'tt0120689',
 'tt0076759',
 'tt0103064',
 'tt0088763',
 'tt0245429',
 'tt0253474',
 'tt0054215',
 'tt6751668',
 'tt0110413',
 'tt0110357',
 'tt0172495',
 'tt0120586',
 'tt0407887',
 'tt0114814',
 'tt0482571',
 'tt2582802',
 'tt0034583',
 'tt0095327',
 'tt0056058',
 'tt1675434',
 'tt0027977',
 'tt0064116',
 'tt0095765',
 'tt0047396',
 'tt0078748',
 'tt0021749',
 'tt0078788',
 'tt0209144',
 'tt1853728',
 'tt0082971',
 'tt0910970',
 'tt0405094',
 'tt0043014',
 'tt0050825',
 'tt0081505',
 'tt0032553',
 'tt4154756',
 'tt0051201',
 'tt0090605',
 'tt4633694',
 'tt0169547',
 'tt0057012',
 'tt1345836',
 'tt0364569',
 'tt0361748',
 'tt00

In [7]:
len(unique_key)

624

In [8]:
my_set = set(t_list)
len(my_set)

624

In [9]:
responses = []
for i in tqdm(unique_key):
    responses += [requests.request(url=f"http://www.omdbapi.com/?i={i}&r=json&apikey=14f99d9e",method="GET")]

100%|████████████████████████████████████████████████████████████████████████████████| 624/624 [01:56<00:00,  5.35it/s]


In [10]:
responses

[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [

In [15]:
responses[0].json()

{'Title': 'The Shawshank Redemption',
 'Year': '1994',
 'Rated': 'R',
 'Released': '14 Oct 1994',
 'Runtime': '142 min',
 'Genre': 'Drama',
 'Director': 'Frank Darabont',
 'Writer': 'Stephen King, Frank Darabont',
 'Actors': 'Tim Robbins, Morgan Freeman, Bob Gunton',
 'Plot': 'Over the course of several years, two convicts form a friendship, seeking consolation and, eventually, redemption through basic compassion.',
 'Language': 'English',
 'Country': 'United States',
 'Awards': 'Nominated for 7 Oscars. 21 wins & 43 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '9.3/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '91%'},
  {'Source': 'Metacritic', 'Value': '81/100'}],
 'Metascore': '81',
 'imdbRating': '9.3',
 'imdbVotes': '2,704,541',
 'imdbID': 'tt0111161',
 'Type': 'movie',
 'DVD': '21 Dec 1999',
 'BoxOffice

In [16]:
df_responses = pd.DataFrame(columns=responses[0].json().keys())

for r in responses:
    df_responses.loc[df_responses.shape[0]] = r.json()

df_responses

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,Drama,Frank Darabont,"Stephen King, Frank Darabont","Tim Robbins, Morgan Freeman, Bob Gunton","Over the course of several years, two convicts...",...,81,9.3,"2,704,541",tt0111161,movie,21 Dec 1999,"$28,767,189",N/A,N/A,True
1,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola","Marlon Brando, Al Pacino, James Caan",The aging patriarch of an organized crime dyna...,...,100,9.2,"1,880,874",tt0068646,movie,11 May 2004,"$136,381,073",N/A,N/A,True
2,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan, Christopher Nolan, David S. Goyer","Christian Bale, Heath Ledger, Aaron Eckhart",When the menace known as the Joker wreaks havo...,...,84,9.0,"2,682,004",tt0468569,movie,09 Dec 2008,"$534,987,076",N/A,N/A,True
3,The Godfather Part II,1974,R,18 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola, Mario Puzo","Al Pacino, Robert De Niro, Robert Duvall",The early life and career of Vito Corleone in ...,...,90,9.0,"1,284,201",tt0071562,movie,24 May 2005,"$47,834,595",N/A,N/A,True
4,12 Angry Men,1957,Approved,10 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,Reginald Rose,"Henry Fonda, Lee J. Cobb, Martin Balsam",The jury in a New York City murder trial is fr...,...,97,9.0,"798,687",tt0050083,movie,04 Mar 2008,N/A,"Criterion Collection, MGM/UA Home Entertainmen...",N/A,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,Yahsi Bati,2009,N/A,01 Jan 2010,112 min,"Adventure, Comedy, Fantasy",Ömer Faruk Sorak,Cem Yilmaz,"Cem Yilmaz, Ozan Güven, Demet Evgar","In 1881, two Ottoman Secret Agents travel to t...",...,N/A,7.4,"36,518",tt1567448,movie,N/A,N/A,N/A,N/A,True
620,The Ballad of Buster Scruggs,2018,R,09 Nov 2018,133 min,"Comedy, Drama, Musical","Ethan Coen, Joel Coen","Joel Coen, Ethan Coen, Jack London","Tim Blake Nelson, Willie Watson, Clancy Brown",Six tales of life and violence in the Old West...,...,79,7.3,"153,067",tt6412452,movie,16 Nov 2018,N/A,N/A,N/A,True
621,Pale Rider,1985,R,28 Jun 1985,115 min,"Drama, Western",Clint Eastwood,"Michael Butler, Dennis Shryack","Clint Eastwood, Michael Moriarty, Carrie Snodg...",A mysterious preacher (Clint Eastwood) protect...,...,61,7.3,"61,176",tt0089767,movie,18 Nov 1997,"$41,410,568","Warner Bros., Warner Home Vídeo",N/A,True
622,Seven Brides for Seven Brothers,1954,Approved,06 Aug 1954,102 min,"Comedy, Drama, Musical",Stanley Donen,"Albert Hackett, Frances Goodrich, Dorothy King...","Jane Powell, Howard Keel, Jeff Richards","In 1850 Oregon, when a backwoodsman brings a w...",...,75,7.3,"25,783",tt0047472,movie,27 Apr 1999,N/A,Metro-Goldwyn-Mayer,N/A,True


In [22]:
films = df_responses.copy()

In [25]:
films.Writer

0                           Stephen King, Frank Darabont
1                       Mario Puzo, Francis Ford Coppola
2      Jonathan Nolan, Christopher Nolan, David S. Goyer
3                       Francis Ford Coppola, Mario Puzo
4                                          Reginald Rose
                             ...                        
619                                           Cem Yilmaz
620                   Joel Coen, Ethan Coen, Jack London
621                       Michael Butler, Dennis Shryack
622    Albert Hackett, Frances Goodrich, Dorothy King...
623                                            Nick Cave
Name: Writer, Length: 624, dtype: object

In [77]:
films

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,[Drama],Frank Darabont,"[Stephen King, Frank Darabont]","[Tim Robbins, Morgan Freeman, Bob Gunton]","Over the course of several years, two convicts...",...,81,9.3,2704541,tt0111161,movie,21 Dec 1999,28767189,N/A,N/A,True
1,The Godfather,1972,R,24 Mar 1972,175 min,"[Crime, Drama]",Francis Ford Coppola,"[Mario Puzo, Francis Ford Coppola]","[Marlon Brando, Al Pacino, James Caan]",The aging patriarch of an organized crime dyna...,...,100,9.2,1880874,tt0068646,movie,11 May 2004,136381073,N/A,N/A,True
2,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"[Action, Crime, Drama]",Christopher Nolan,"[Jonathan Nolan, Christopher Nolan, David S. G...","[Christian Bale, Heath Ledger, Aaron Eckhart]",When the menace known as the Joker wreaks havo...,...,84,9.0,2682004,tt0468569,movie,09 Dec 2008,534987076,N/A,N/A,True
3,The Godfather Part II,1974,R,18 Dec 1974,202 min,"[Crime, Drama]",Francis Ford Coppola,"[Francis Ford Coppola, Mario Puzo]","[Al Pacino, Robert De Niro, Robert Duvall]",The early life and career of Vito Corleone in ...,...,90,9.0,1284201,tt0071562,movie,24 May 2005,47834595,N/A,N/A,True
4,12 Angry Men,1957,Approved,10 Apr 1957,96 min,"[Crime, Drama]",Sidney Lumet,[Reginald Rose],"[Henry Fonda, Lee J. Cobb, Martin Balsam]",The jury in a New York City murder trial is fr...,...,97,9.0,798687,tt0050083,movie,04 Mar 2008,0,"Criterion Collection, MGM/UA Home Entertainmen...",N/A,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,Yahsi Bati,2009,N/A,01 Jan 2010,112 min,"[Adventure, Comedy, Fantasy]",Ömer Faruk Sorak,[Cem Yilmaz],"[Cem Yilmaz, Ozan Güven, Demet Evgar]","In 1881, two Ottoman Secret Agents travel to t...",...,0,7.4,36518,tt1567448,movie,N/A,0,N/A,N/A,True
620,The Ballad of Buster Scruggs,2018,R,09 Nov 2018,133 min,"[Comedy, Drama, Musical]","Ethan Coen, Joel Coen","[Joel Coen, Ethan Coen, Jack London]","[Tim Blake Nelson, Willie Watson, Clancy Brown]",Six tales of life and violence in the Old West...,...,79,7.3,153067,tt6412452,movie,16 Nov 2018,0,N/A,N/A,True
621,Pale Rider,1985,R,28 Jun 1985,115 min,"[Drama, Western]",Clint Eastwood,"[Michael Butler, Dennis Shryack]","[Clint Eastwood, Michael Moriarty, Carrie Snod...",A mysterious preacher (Clint Eastwood) protect...,...,61,7.3,61176,tt0089767,movie,18 Nov 1997,41410568,"Warner Bros., Warner Home Vídeo",N/A,True
622,Seven Brides for Seven Brothers,1954,Approved,06 Aug 1954,102 min,"[Comedy, Drama, Musical]",Stanley Donen,"[Albert Hackett, Frances Goodrich, Dorothy Kin...","[Jane Powell, Howard Keel, Jeff Richards]","In 1850 Oregon, when a backwoodsman brings a w...",...,75,7.3,25783,tt0047472,movie,27 Apr 1999,0,Metro-Goldwyn-Mayer,N/A,True


In [28]:
films["Writer"] = films.Writer.apply(lambda x: x.split(', '))
films["Actors"] = films.Actors.apply(lambda x: x.split(', '))

In [31]:
films["Genre"] = films.Genre.apply(lambda x: x.split(', '))

In [35]:
films["Year"] = films.Year.apply(lambda x: int(x[0:4]))

In [44]:
films["Metascore"] = films.Metascore.apply(lambda x: 0 if x=="N/A" else int(x))

In [46]:
films["imdbRating"] = films.imdbRating.apply(lambda x: float(x))

In [61]:
films["BoxOffice"] = films.BoxOffice.apply(lambda x: str(x))

In [69]:
films["BoxOffice"] = films.BoxOffice.apply(lambda x: 0 if x=="N/A" else ( 0 if x=="nan" else x.replace(',','')))

In [76]:
films["BoxOffice"] = films.BoxOffice.apply(lambda x: int(x.replace("$","") if isinstance(x, str) == True))

In [70]:
films.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624 entries, 0 to 623
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Title       624 non-null    object 
 1   Year        624 non-null    int64  
 2   Rated       624 non-null    object 
 3   Released    624 non-null    object 
 4   Runtime     624 non-null    object 
 5   Genre       624 non-null    object 
 6   Director    624 non-null    object 
 7   Writer      624 non-null    object 
 8   Actors      624 non-null    object 
 9   Plot        624 non-null    object 
 10  Language    624 non-null    object 
 11  Country     624 non-null    object 
 12  Awards      624 non-null    object 
 13  Poster      624 non-null    object 
 14  Ratings     624 non-null    object 
 15  Metascore   624 non-null    int64  
 16  imdbRating  624 non-null    float64
 17  imdbVotes   624 non-null    int64  
 18  imdbID      624 non-null    object 
 19  Type        624 non-null    o

In [78]:
films.to_json('films.json',orient="records", compression = 'infer', indent=4)

In [20]:
df_responses["Ratings"]

0      [{'Source': 'Internet Movie Database', 'Value'...
1      [{'Source': 'Internet Movie Database', 'Value'...
2      [{'Source': 'Internet Movie Database', 'Value'...
3      [{'Source': 'Internet Movie Database', 'Value'...
4      [{'Source': 'Internet Movie Database', 'Value'...
                             ...                        
619    [{'Source': 'Internet Movie Database', 'Value'...
620    [{'Source': 'Internet Movie Database', 'Value'...
621    [{'Source': 'Internet Movie Database', 'Value'...
622    [{'Source': 'Internet Movie Database', 'Value'...
623    [{'Source': 'Internet Movie Database', 'Value'...
Name: Ratings, Length: 624, dtype: object

In [47]:
df_responses["Actors"][1].split(",")
actor_set = set()
#actor_list = []
actorsplit = ""

for actors in df_responses["Actors"]:
    actorsplit = actors.split(", ")
    for actor in actorsplit:
        #print(actor)
        actor_set.add(actor)
        #actor_list.append(actor)

writer_set = set()
writersplit = ""

for writers in df_responses["Writer"]:
    writersplit = writers.split(", ")
    for writer in writersplit:
        #print(actor)
        writer_set.add(writer)
        #actor_list.append(actor)

director_set = set()
directorsplit = ""

for directors in df_responses["Director"]:
    directorsplit = directors.split(", ")
    for director in directorsplit:
        #print(actor)
        director_set.add(director)
        #actor_list.append(actor)


#print(len(actor_list))
print(len(actor_set))
print(len(writer_set))
print(len(director_set))

1376
1088
448


In [44]:
actor_list = list(actor_set)
actor_list

['Dev Patel',
 'Charlton Heston',
 'Tim McGraw',
 'Dana Andrews',
 'Robert Blake',
 'Friedrich Feher',
 'Daniel Mays',
 'Bradley Cooper',
 'Everett Sloane',
 'Subir Banerjee',
 'Elizabeth McGovern',
 'Tessa Thompson',
 'Rajit Kapoor',
 'Emily Mortimer',
 'Mahie Gill',
 'Craig T. Nelson',
 'Dustin Hoffman',
 'Jean Heywood',
 'Ruth Roman',
 'Mark Gatiss',
 'Jason Mitchell',
 'Nick Nolte',
 'Deepika Padukone',
 'Kelvin Harrison Jr.',
 'Herbert Grönemeyer',
 'Lori Petty',
 'Darshan Kumaar',
 'Gloria Swanson',
 'Ma Dong-seok',
 'Charlie Hunnam',
 'Mel Gibson',
 'Sondra Locke',
 'Lina Leandersson',
 'Rajkummar Rao',
 'Manoj Pahwa',
 'Dean-Charles Chapman',
 'Jacob Tremblay',
 'Cho Jin-woong',
 'Keanu Reeves',
 'Joanne Dru',
 'Judy Garland',
 'Geena Davis',
 'Dulquer Salmaan',
 'Alain Delon',
 'Ian McKellen',
 'Mahito Tsujimura',
 'Deborah Kara Unger',
 'Kirk Douglas',
 'John Getz',
 'Zendaya',
 'Amir Farrokh Hashemian',
 'Margarita Terekhova',
 'Jeff Goldblum',
 'John Turturro',
 'Sô Yamamur

# Importing json

In [81]:
import json

In [82]:
with open("films.json") as f:
    data = json.load(f)

In [96]:
data[0]

{'Title': 'The Shawshank Redemption',
 'Year': 1994,
 'Rated': 'R',
 'Released': '14 Oct 1994',
 'Runtime': '142 min',
 'Genre': ['Drama'],
 'Director': 'Frank Darabont',
 'Writer': ['Stephen King', 'Frank Darabont'],
 'Actors': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'],
 'Plot': 'Over the course of several years, two convicts form a friendship, seeking consolation and, eventually, redemption through basic compassion.',
 'Language': 'English',
 'Country': 'United States',
 'Awards': 'Nominated for 7 Oscars. 21 wins & 43 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '9.3/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '91%'},
  {'Source': 'Metacritic', 'Value': '81/100'}],
 'Metascore': 81,
 'imdbRating': 9.3,
 'imdbVotes': 2704541,
 'imdbID': 'tt0111161',
 'Type': 'movie',
 'DVD': '21 Dec 1999',
 'BoxOffi

In [99]:
data[0]["Actors"][0]

'Tim Robbins'

In [111]:
#Defining the namespace
nspace = Namespace("Project.rdf")

# Create a Graph
g = Graph()

# Parse in an RDF file hosted locally
g.parse("projet3.ttl")

g.namespace_manager.bind("nl",nspace)

# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in g:
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in g:
       raise Exception("It better be!")

# Print the number of "triples" in the Graph
print(f"Graph g has {len(g)} statements.")

# Print out the entire Graph in the RDF Turtle format
print(g.serialize(format="ttl"))

Graph g has 231 statements.
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix webSem: <http://www.semanticweb.org/marya/ontologies/2023/2/webSem#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.semanticweb.org/marya/ontologies/2023/2/webSem> a owl:Ontology .

webSem:Actor a owl:Class ;
    rdfs:subClassOf webSem:Person ;
    owl:equivalentClass [ a owl:Class ;
            owl:intersectionOf ( webSem:Person [ a owl:Restriction ;
                        owl:onProperty webSem:isActorOf ;
                        owl:someValuesFrom webSem:Movie ] ) ] .

webSem:BTime2 a webSem:BroadcastTime,
        owl:NamedIndividual ;
    webSem:endDate "2023-03-20T00:00:00"^^xsd:dateTime ;
    webSem:startDate "2023-03-08T00:00:00"^^xsd:dateTime .

webSem:DanielCraig a webSem:Person,
        owl:NamedIndividual ;
    webSem:age "55"^^xsd:int ;
    webSem:isActorOf 

In [112]:
for s, p, o in g:
    print(s,p,o)

http://www.semanticweb.org/marya/ontologies/2023/2/webSem#isDirectorOf http://www.w3.org/2000/01/rdf-schema#range http://www.semanticweb.org/marya/ontologies/2023/2/webSem#Movie
http://www.semanticweb.org/marya/ontologies/2023/2/webSem#name http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#DatatypeProperty
n4a5ca906dede4517a7f661f77b4cac30b6 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Restriction
http://www.semanticweb.org/marya/ontologies/2023/2/webSem#inTheDatesOf http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
http://www.semanticweb.org/marya/ontologies/2023/2/webSem#inTheDatesOf http://www.w3.org/2000/01/rdf-schema#range http://www.semanticweb.org/marya/ontologies/2023/2/webSem#BroadcastTime
http://www.semanticweb.org/marya/ontologies/2023/2/webSem#PulpFiction http://www.semanticweb.org/marya/ontologies/2023/2/webSem#language English
http://www.semanticweb.org/marya/ontologies

In [116]:
properties = set()

for s, p, o in g:
    properties.add(s)

for p in properties:
    print(p)

http://www.semanticweb.org/marya/ontologies/2023/2/webSem#runtime
http://www.semanticweb.org/marya/ontologies/2023/2/webSem#BTime2
http://www.semanticweb.org/marya/ontologies/2023/2/webSem#isActorOf
http://www.semanticweb.org/marya/ontologies/2023/2/webSem#UmaThurman
http://www.semanticweb.org/marya/ontologies/2023/2/webSem#isProjecting
n4a5ca906dede4517a7f661f77b4cac30b8
http://www.semanticweb.org/marya/ontologies/2023/2/webSem#wasDirectedBy
http://www.semanticweb.org/marya/ontologies/2023/2/webSem#isDirectorOf
http://www.semanticweb.org/marya/ontologies/2023/2/webSem#name
http://www.semanticweb.org/marya/ontologies/2023/2/webSem#metacriticRating
http://www.semanticweb.org/marya/ontologies/2023/2/webSem#UGCLesHalles
n4a5ca906dede4517a7f661f77b4cac30b12
http://www.semanticweb.org/marya/ontologies/2023/2/webSem#tmdbVoteCount
n4a5ca906dede4517a7f661f77b4cac30b6
http://www.semanticweb.org/marya/ontologies/2023/2/webSem#BroadcastTime
http://www.semanticweb.org/marya/ontologies/2023/2/webSe

In [117]:
for prefix, ns in g.namespaces():
    print(prefix, ns)

owl http://www.w3.org/2002/07/owl#
rdf http://www.w3.org/1999/02/22-rdf-syntax-ns#
rdfs http://www.w3.org/2000/01/rdf-schema#
xsd http://www.w3.org/2001/XMLSchema#
xml http://www.w3.org/XML/1998/namespace
 http://www.semanticweb.org/marya/ontologies/2023/2/untitled-ontology-9/
webSem http://www.semanticweb.org/marya/ontologies/2023/2/webSem#
untitled-ontology-9 http://www.semanticweb.org/marya/ontologies/2023/2/untitled-ontology-9#
nl Project.rdf


In [ ]:
g.add()